# Gaussian Regression of Light-Curves

This notebook explores the use of a Gaussian regression to fit CSP light curves. We use a modified version of the regression implimentation from the `avocado` package by [Kyle Boone](https://github.com/kboone).

#### Analysis steps considered by this notebook:

1. <a href='#gaussian_regression'>Gaussian Regression</a>: Fit observed light curves using a Gaussian regression
1. <a href='#err_relationship'>Measured and Regressed Error</a>: Examine the relationship between error values reported by CSP and the Gaussian regression


In [ ]:
import sys
from copy import copy

import numpy as np
import sncosmo
from IPython.display import clear_output
from astropy.table import Column, hstack, vstack
from astropy.table import Table
from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator
from sndata.csp import DR3

sys.path.insert(0, '../')
from analysis import utils
from analysis import regression

dr3 = DR3()
dr3.download_module_data()
dr3.register_filters(force=True)


# Gaussian Regression <a id='gaussian_regression'></a>

We use Gaussian regression to fit the light curve in each band. These fits are then compared against a simultaneous fit performed using the Hsiao model.


In [ ]:
def plot_regression(data, time, bands, pred_flux, pred_unc):
    """Plot regression results in flux space
    
    Plots data['flux'] and data['fluxerr'] vs time in each band.
    Plots pred_flux and pred_unc vs time in each band.
    
    Args:
        data            (Table): The measured photometric data
        pred_flux (list[float]): Predicted flux from the regression
        pred_unc  (list[float]): Flux uncertainty from the regression
    
    Returns:
        The figure object
        An array of figure axes
    """

    columns = 2
    rows = 5
    fig_shape = (rows, columns)
    fig_size = (3 * rows, 6 * columns)

    fig, axes = plt.subplots(*fig_shape, figsize=fig_size, sharey=True)
    for axis in axes[rows - 1]:
        axis.set_xlabel('Time')

    for axis in axes.flatten()[::2]:
        axis.set_ylabel('Flux')

    plot_data = zip(bands, axes.flatten(), pred_flux, pred_unc)
    for band_name, axis, f, fe in plot_data:
        band_data = data[data['band'] == band_name]
        axis.scatter(band_data['time'], band_data['flux'], label=band_name)
        axis.errorbar(band_data['time'], band_data['flux'],
                      band_data['fluxerr'], linestyle='', label=None)

        axis.plot(time, f, linestyle='--', color='black', label='Regression')
        axis.fill_between(
            time,
            f - fe,
            f + fe,
            alpha=.3,
            color='grey',
            label='Regression Error'
        )

        axis.xaxis.set_minor_locator(MultipleLocator(1))
        axis.set_xlim(min(time), max(time))
        axis.legend()

    plt.tight_layout()
    return fig, axes


def subplot_sncosmo_flux(time, bands, fitted_model, axes):
    """Plot the predicted flux of an sncosmo in each band vs time
    
    Args:
        fitted_model (Model): An SNCosmo model fitted to the data
        axes         (array): Use existing axes
    """

    for band, axis in zip(bands, axes.flatten()):
        try:
            band_flux = fitted_model.bandflux(band, time)

        except ValueError as e:
            print(
                f'{fitted_model.source.name} {fitted_model.source.version}: {e}')
            pass

        else:
            axis.plot(time, band_flux, label=f'{fitted_model.source.name}')
            axis.legend()


def subplot_sncosmo_mag(time, bands, fitted_model, axes):
    """Plot the predicted magnitude of an sncosmo in each band vs time
    
    Uses a zero point of 25.
    
    Args:
        fitted_model (Model): An SNCosmo model fitted to the data
        axes         (array): Use existing axes
    """

    for band, axis in zip(bands, axes.flatten()):
        try:
            band_flux = -2.5 * np.log10(fitted_model.bandflux(band, time)) + 25

        except ValueError as e:
            print(f'{fitted_model.source.name} {fitted_model.source.version}: {e}')
            pass

        else:
            axis.plot(time, band_flux, label=f'{fitted_model.source.name}')
            axis.legend()


First, we consider the fit in flux space.

In [ ]:
def plot_dr3_object_flux(obj_id):
    """Plot the regression and Hsiao model fit for an object in flux space
    
    Args:
        obj_id (str): The Id of the object to plot
        
    Returns:
        The figure object
        An array of figure axes
    """

    data = dr3.get_data_for_id(obj_id)

    # Fit hsiao
    sncosmo_model = sncosmo.Model('hsiao')
    sncosmo_model.set(z=data.meta['z'])
    sncosmo_model.set(t0=utils.get_csp_t0(obj_id))
    fit_result, fitted_model = sncosmo.fit_lc(data, sncosmo_model, ['amplitude'])

    # Fit a gaussian regression
    gp = regression.fit_gaussian_process(data)
    bands = set(data['band'])
    time = np.arange(min(data['time']) - 1, max(data['time']) + 1)
    gp_flux, gp_unc = regression.predict_light_curve(gp, bands, time)

    # Plot modeled flux
    fig, axes = plot_regression(data, time, bands, gp_flux, gp_unc)
    subplot_sncosmo_flux(time, bands, fitted_model, axes)
    axes[0, 0].set_ylim(0)
    fig.suptitle(obj_id, fontsize=30, y=1.05)

    return fig, axes


In [ ]:
_ = plot_dr3_object_flux('2004ef')
_ = plot_dr3_object_flux('2008bc')


Next we consider the fit in terms of magnitudes.

In [ ]:
def plot_dr3_object_mag(obj_id):
    """Plot the regression and Hsiao model fit for an object in magnitude space
    
    Args:
        obj_id (str): The Id of the object to plot
        
    Returns:
        The figure object
        An array of figure axes
    """

    data = dr3.get_data_for_id(obj_id)

    # Fit hsiao
    sncosmo_model = sncosmo.Model('hsiao')
    sncosmo_model.set(z=data.meta['z'])
    sncosmo_model.set(t0=utils.get_csp_t0(obj_id))
    fit_result, fitted_model = sncosmo.fit_lc(data, sncosmo_model, ['amplitude'])

    data['flux'] = -2.5 * np.log10(data['flux']) + 25
    data['fluxerr'] = (-2.5 * data['fluxerr']) / (data['flux'] * np.log(10))

    # Fit a gaussian regression
    gp = regression.fit_gaussian_process(data)
    bands = set(data['band'])
    time = np.arange(min(data['time']) - 1, max(data['time']) + 1)
    gp_flux, gp_unc = regression.predict_light_curve(gp, bands, time)

    # Plot fitted magnitudes
    fig, axes = plot_regression(data, time, bands, gp_flux, gp_unc)
    subplot_sncosmo_mag(time, bands, fitted_model, axes)
    for axis in axes.flatten()[::2]:
        axis.set_ylabel('Magnitude')

    fig.suptitle(obj_id, fontsize=30, y=1.05)

    return fig, axes


In [ ]:
_ = plot_dr3_object_mag('2004ef')
_ = plot_dr3_object_mag('2008bc')


Although the Gaussian regression performs well within the observed time range, Gaussian regressions are generally bad at extrapolating. For this reason, we choose to limit our use of the above regression when predicting flux values to within the first and last observed data point in each band. 

The regression also tends to drift towards zero at late times when the photometric sampling is low. This results in a non-physical jump in the estimated magnitude.

## Measured and Regressed Errors <a id='err_relationship'></a>

We begin by comparing error values published by CSP against those returned by our implementation of a Gaussian regression. We do this for all SNe included in the CSP DR3 data release.



In [ ]:
def create_regression_table(data_table):
    """Fit photometric observations with a gaussian regression
    
    This function is a wrapper combining the ``fit_gaussian_process``
    and ``predict_light_curve`` functions from the ``regression`` module.

    Args:
        data (Table): Data table from sndata (format_sncosmo=True)

    Returns:
        An astropy table with measured data and regression results
    """

    # Regress light curve and return fitted flux
    bands = set(data_table['band'])
    times = [data_table[data_table['band'] == band]['time'] for band in bands]
    gp = regression.fit_gaussian_process(data_table)
    flux, flux_err = regression.predict_light_curve(gp, bands, times)

    pred_tables = []
    for b, t, f, fe in zip(bands, times, flux, flux_err):
        table_this = Table([t, f, fe], names=['time', 'reg_flux', 'reg_fluxerr'])
        table_this['band'] = Column(b, dtype=object, length=len(table_this))
        pred_tables.append(table_this)

    # Use hstack instead of join since there are duplicate band / time combos
    data_table = copy(data_table)
    data_table.sort(['band', 'time'])
    prediction = vstack(pred_tables)
    prediction.sort(['band', 'time'])
    prediction.remove_columns(['band', 'time'])
    out_table = hstack([data_table, prediction])

    out_table['obj_id'] = data_table.meta['obj_id']
    out_table['residuals'] = out_table['flux'] - out_table['reg_flux']
    return out_table


def plot_reg_errors(flux, flux_err, reg_err):
    """Plot regressed vs published error values 
    
    Args:
        reg_results (Table): A table from ``create_regression_table``
        
    Returns:
        The figure object
        An array of figure axes
    """

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    plot_combos = (
        (flux, flux_err),
        (flux, reg_err),
        (flux_err, reg_err)
    )

    for (x, y), axis in zip(plot_combos, axes):
        axis.scatter(x, y, label=None)

        fit_params = np.polyfit(x, y, 1)
        fit_func = np.poly1d(fit_params)
        axis.plot(
            x, fit_func(x),
            label=f'{fit_params[0]:0.2}x + {fit_params[1]:0.2}',
            color='black'
        )

        axis.set_xlim(min(x), max(x))
        axis.set_ylim(min(y), max(y))
        axis.legend()

    axes[0].set_xlabel(f'Published Flux')
    axes[0].set_ylabel(f'Published Error')
    axes[1].set_xlabel(f'Published Flux')
    axes[1].set_ylabel(f'Regression Error')
    axes[2].set_xlabel(f'Published Error')
    axes[2].set_ylabel(f'Regression Error')

    axes[2].plot([-1, 1], [-1, 1], color='black', linestyle='--')

    plt.tight_layout()
    plt.show()

    return fig, axes


In [ ]:
all_targets = vstack(
    [create_regression_table(data_table) for data_table in dr3.iter_data(verbose=True)],
    metadata_conflicts='silent'
)


In [ ]:
fig, axes = plot_reg_errors(all_targets['flux'], all_targets['fluxerr'], all_targets['reg_fluxerr'])
fig.suptitle('All CSP DR1 Targets')
plt.show()


We note that the regression errors are about 60% the size of the published errors and are always smaller. However, this plot is misleading as the relationship in the right-most sub plot is not actually linear. When we inspect the results of a single object, we see that the trend is much more like a negative exponential.

In [ ]:
def plot_resampled_errors(obj_id, scale_by='all'):
    """Plot resampled error vs published and single regression errors
    
    Args:
        reg_results (Table): An initial regression from ``create_regression_table``
        
    Returns:
        A matplotlib figure
        An array of matplotlib axes
    """

    # Perform a gaussian regression on the data
    data = dr3.get_data_for_id(obj_id)
    reg_results = create_regression_table(data)
    reg_results.sort('fluxerr')
    x = reg_results['fluxerr']
    y = reg_results['reg_fluxerr']

    # setup figure
    fig, axes = plt.subplots(
        2, 1,
        figsize=(6, 6),
        gridspec_kw={'height_ratios': [3, 1]},
        sharex=True)

    limit = max(max(x), max(y))
    axes[0].plot([0, limit], [0, limit], label=None, linestyle='--', color='k')
    axes[0].set_xlim(0, limit)
    axes[0].set_ylim(0, limit)
    axes[0].set_ylabel(f'Regression Error')
    axes[0].set_title(obj_id)
    axes[1].set_xlabel(f'Published Error')
    axes[1].set_ylabel('Residuals (%)', labelpad=25)
    plt.subplots_adjust(hspace=0)

    # Scale the error using a negative exponential
    if scale_by == 'all':
        scaled_errors, exp_opt, exp_cov = regression.scale_errors(x, y)
        exp_yvals = regression.neg_exponential(x, *exp_opt)
        axes[0].plot(x, exp_yvals, color='k', zorder=100)

    # Plot the errors in each band
    for band in set(reg_results['band']):
        indices = np.where(reg_results['band'] == band)[0]
        band_x, band_y = x[indices], y[indices]
        axes[0].scatter(band_x, band_y, s=10)

        if scale_by == 'band':
            band_scaled_errors, exp_opt, exp_cov = regression.scale_errors(band_x, band_y)
            exp_yvals = regression.neg_exponential(x, *exp_opt)
            axes[0].plot(x, exp_yvals, alpha=.5)

        residuals = (band_y - regression.neg_exponential(band_x, *exp_opt)) / band_y
        axes[1].scatter(band_x, residuals, s=10)

    return fig, axes


In [ ]:
plot_resampled_errors('2005kc')
plot_resampled_errors('2005kc', 'band')
plot_resampled_errors('2004ef')
plot_resampled_errors('2004ef', 'band')
